<a href="https://colab.research.google.com/github/FreNzQuiN/Collab_Stable-Diffusion/blob/main/newInlineSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q --upgrade diffusers invisible_watermark transformers accelerate safetensors compel
import torch
from torch import autocast
from diffusers import StableDiffusionXLPipeline, DDIMScheduler #EulerAncestralDiscreteScheduler,

model = "linaqruf/animagine-xl"

pipe = StableDiffusionXLPipeline.from_pretrained(
    model,
    torch_dtype=torch.float16,
    use_safetensors=True,
    safety_checker = None,
    requires_safety_checker = False,
    variant="fp16",
    )
pipe.scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipe.to('cuda')


In [ ]:
num=0
def disabled_safety_checker(images, clip_input):
    if len(images.shape)==4:
        num_images = images.shape[0]
        return images, [False]*num_images
    else:
        return images, False
pipe.safety_checker = disabled_safety_checker

In [ ]:
from compel import Compel, ReturnedEmbeddingsType
import matplotlib.pyplot as plt
from PIL import Image
import random
import torch, gc

prompt = "anime girl white hair, vivid color, art splash"
negative_prompt = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"
outputbase = "/content/anime_girl"
compel = Compel(
  tokenizer=[pipe.tokenizer, pipe.tokenizer_2] ,
  text_encoder=[pipe.text_encoder, pipe.text_encoder_2],
  returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
  requires_pooled=[False, True]
)
conditioning, pooled = compel(prompt)
W = 1024
H = 1424
setseed = random.randint(0,1000000)

for x in range(1):
  gc.collect()
  torch.cuda.empty_cache()
  image = pipe(
      #prompt_embeds=conditioning, pooled_prompt_embeds=pooled,
      prompt=prompt,
      negative_prompt=negative_prompt,
      seed=setseed,
      width=W,
      height=H,
      guidance_scale=6,
      guidance_rescale=0.9,
      target_size=(W,H),
      original_size=(4096,4096),
      num_inference_steps=25
      ).images[0]
  output=f"{outputbase}{num}_{setseed}.png"
  image.save(output)
  image = Image.open(output)
  plt.imshow(image)
  num=num+1
gc.collect()
torch.cuda.empty_cache()


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!pip install numba
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
!rm "/content/"*.png

In [ ]:
#@title (STEP 1) Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

TheDate = "2024-12-04" #@param {type:'string'}
!mv "/content/outputs/" "/content/drive/MyDrive/"{Thedate}/

In [ ]:
!mkdir outputs

In [ ]:
!mv "/content/"*.png "/content/outputs"

In [ ]:
!smi-nvidia